In [1]:
# Import the data
qiime_taxonomy = '/Users/claranordquist/Documents/Universitetet/HT24/Tillämpad_bioinformatik/Applied-bioinformatics/05_Plot_the_results/01_Data/Fasta_short_reads/Fasta_short_reads.tsv'
output_taxonomy = '/Users/claranordquist/Documents/Universitetet/HT24/Tillämpad_bioinformatik/Applied-bioinformatics/05_Plot_the_results/01_Data/Fasta_short_reads/Fasta_short_reads_fixed.csv'

# Import packages
import pandas as pd
import numpy as np
import regex as re

In [2]:
# Read the input data
qiime_read = pd.read_csv(qiime_taxonomy, sep=' ', skiprows=1, header=None)
qiime_read.columns = ['Feature ID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Genus1', 'Species', 'Confidence']
qiime_read.set_index('Feature ID', inplace=True)


In [3]:
qiime_copy = qiime_read.copy(deep=True)
# Loop through the whole dataframe
for column in qiime_copy.columns[1:-2]:
    for row in qiime_copy.index:
        value = qiime_copy.loc[row, column]
        # Check if value is a string before searching for '__'
        if isinstance(value, str) and '__' not in value:
            qiime_copy.loc[row, 'Confidence'] = value
            qiime_copy.loc[row, column] = np.NaN

for column in qiime_copy.columns[:-1]:
    for row in qiime_copy.index:
        value = qiime_copy.loc[row, column]
        if isinstance(value, str):
            qiime_copy.loc[row, column] = value.replace(';', '').strip()

qiime_copy['Confidence'] = qiime_copy['Confidence'].apply(lambda x: float(x))

In [4]:
qiime_copy.drop('Genus1', axis=1, inplace=True)

In [5]:
# Delete the taxonomical levels from the names (k__Bacteria --> Bacteria)
def delete_prefix(name):
    '''Deletes the taxonomical prefix in classifications
    k_Bacteria --> Bacteria'''
    if type(name) == str:
        return re.sub('.{1}__', '', name)

# Take away things following _ (Akkermansia muciniphila_D_776786 --> Akkermansia muciniphila)
def extract_name(name):
    '''Deletes all characters following _
    Akkermansia muciniphila_D_776786 --> Akkermansia muciniphila'''
    if type(name) == str:
        return re.sub(r'_.*', '', name)

In [6]:
a = qiime_copy.copy(deep=True)

In [7]:
for column in qiime_copy.columns:
    for row in qiime_copy.index:
        value = delete_prefix(a.loc[row, column])
        name = extract_name(value)
        a.loc[row, column] = name

In [8]:
a.to_csv(output_taxonomy)